In [9]:
import wikipedia
import spacy
import re
import gradio as gr

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    """
    Extract named entities using spaCy NLP.
    """
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

def wikipedia_fact_check(sentence):
    """
    For a given sentence, extract entities and verify existence in Wikipedia.
    """
    entities = extract_entities(sentence)
    if not entities:
        return "⚠️ No verifiable facts found in this sentence."

    for entity in entities:
        try:
            search_results = wikipedia.search(entity, results=2)
            if not search_results:
                return f"⚠️ Entity '{entity}' not found → Possible hallucination"
        except Exception as e:
            return f"❌ Error: {e}"
    
    return "✔️ All named entities found → Likely factual"

def analyze_text(text):
    """
    Main analysis function: splits input text into sentences,
    checks each sentence via Wikipedia, and returns the result.
    """
    sentences = re.split(r'(?<=[.!?]) +', text)
    output = []

    for i, sentence in enumerate(sentences):
        if not sentence.strip():
            continue
        result = wikipedia_fact_check(sentence)
        output.append(f"Sentence {i+1}: {sentence.strip()}\n→ {result}")

    return "\n\n".join(output)

# Gradio interface
interface = gr.Interface(
    fn=analyze_text,
    inputs=gr.Textbox(lines=10, placeholder="Paste LLM output here..."),
    outputs="text",
    title="🧠 Hallucination Checker for LLM Output",
    description="This tool analyzes each sentence using Wikipedia and highlights potential hallucinations based on named entity verification.",
)

if __name__ == "__main__":
    interface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
